# Agent Search with Amazon OpenSearch

In this module, we'll build an intelligent AI agent that can understand natural language queries and interact with our OpenSearch semantic search system. This represents the next evolution in search technology - from static keyword searches to conversational, context-aware search experiences.

## What We'll Build

1. **Conversational Search Agent**: An AI agent that can understand natural language queries and convert them into effective search strategies
2. **Context-Aware Responses**: The agent maintains conversation context and can handle follow-up questions
3. **Multi-Modal Search**: Combine keyword, semantic, and contextual search based on query analysis
4. **Intelligent Result Synthesis**: Use LLMs to synthesize search results into coherent, helpful responses
5. **Interactive Demo**: A working demonstration of agent-powered search

## Architecture Overview

Our agent search system combines:
- **Language Model**: AWS Bedrock (Claude/Titan) for natural language understanding and response generation
- **Search Engine**: OpenSearch with semantic and keyword search capabilities
- **Agent Framework**: Custom agent logic to orchestrate search and response generation
- **Context Management**: Conversation history and context tracking

### 1. Install Required Libraries

We'll need additional libraries for building our AI agent, including AWS Bedrock SDK and conversation management tools.

In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q sentence-transformers
!pip install -q pandas
!pip install -q numpy
!pip install -q tqdm
# Install additional libraries for agent functionality
!pip install -q langchain
!pip install -q langchain-aws

### 2. Import Required Libraries

In [ ]:
import json
import boto3
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Any, Optional
import time

# OpenSearch and search libraries
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# Sentence transformers for embeddings
from sentence_transformers import SentenceTransformer

# AWS Bedrock for LLM capabilities
from botocore.exceptions import ClientError

print("✅ All required libraries imported successfully!")

### 3. Setup Configuration and CloudFormation Outputs

Let's get the configuration from our CloudFormation stack that was set up in previous modules.

In [ ]:
import boto3

def get_cfn_outputs(stackname):
    """Get CloudFormation stack outputs"""
    cfn = boto3.client('cloudformation')
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

# Setup variables to use for the rest of the demo
cloudformation_stack_name = "static-cloudformation-semantic-search"

try:
    outputs = get_cfn_outputs(cloudformation_stack_name)
    bucket = outputs['s3BucketTraining']
    aos_host = outputs['OpenSearchDomainEndpoint']
    
    print("✅ CloudFormation outputs retrieved successfully!")
    print(f"S3 Bucket: {bucket}")
    print(f"OpenSearch Host: {aos_host}")
except Exception as e:
    print(f"⚠️ Could not retrieve CloudFormation outputs: {e}")
    print("Please ensure your CloudFormation stack is deployed and accessible.")
    # Fallback configuration
    aos_host = "your-opensearch-domain.us-east-1.es.amazonaws.com"
    bucket = "your-s3-bucket"

### 4. Setup AWS Service Connections

We'll establish connections to OpenSearch, Bedrock, and other AWS services needed for our agent.

In [ ]:
# Setup AWS region and credentials
region = 'us-east-1'

# Setup OpenSearch connection
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)

aos_client = OpenSearch(
    hosts=[{'host': aos_host, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Setup Bedrock client for LLM capabilities
bedrock_client = boto3.client('bedrock-runtime', region_name=region)

# Initialize sentence transformer for generating embeddings
try:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    print("✅ Sentence transformer model loaded successfully!")
except Exception as e:
    print(f"⚠️ Could not load sentence transformer: {e}")
    print("Will download model on first use.")

print("✅ AWS service connections established!")

### 5. Build the Search Agent Class

Now we'll create our intelligent search agent that combines natural language understanding with OpenSearch capabilities.

In [ ]:
class SearchAgent:
    """Intelligent Search Agent for OpenSearch with conversational capabilities"""
    
    def __init__(self, opensearch_client, bedrock_client, sentence_model, index_name='nlp_pqa'):
        self.os_client = opensearch_client
        self.bedrock_client = bedrock_client
        self.sentence_model = sentence_model
        self.index_name = index_name
        self.conversation_history = []
        
    def analyze_query_intent(self, query: str) -> Dict[str, Any]:
        """Analyze user query to determine search strategy"""
        
        # Simple heuristic-based intent analysis
        query_lower = query.lower()
        
        intent_analysis = {
            'query': query,
            'search_type': 'semantic',  # Default to semantic search
            'needs_context': False,
            'is_followup': False,
            'extracted_keywords': [],
            'confidence': 0.8
        }
        
        # Check for follow-up patterns
        followup_patterns = ['what about', 'how about', 'what if', 'can you', 'do you', 'more like']
        if any(pattern in query_lower for pattern in followup_patterns) and self.conversation_history:
            intent_analysis['is_followup'] = True
            intent_analysis['needs_context'] = True
        
        # Check for keyword search indicators
        keyword_indicators = ['exact', 'specifically', 'precisely', 'literally']
        if any(indicator in query_lower for indicator in keyword_indicators):
            intent_analysis['search_type'] = 'keyword'
        
        # Check for comparison queries
        comparison_patterns = ['compare', 'difference', 'versus', 'vs', 'better']
        if any(pattern in query_lower for pattern in comparison_patterns):
            intent_analysis['search_type'] = 'hybrid'
        
        return intent_analysis
    
    def perform_semantic_search(self, query: str, k: int = 10) -> List[Dict]:
        """Perform semantic search using vector embeddings"""
        try:
            # Generate embedding for the query
            query_vector = self.sentence_model.encode([query])[0].tolist()
            
            # Perform KNN search
            search_body = {
                'size': k,
                'query': {
                    'knn': {
                        'question_vector': {
                            'vector': query_vector,
                            'k': k
                        }
                    }
                },
                '_source': ['question', 'answer']
            }
            
            response = self.os_client.search(index=self.index_name, body=search_body)
            
            results = []
            for hit in response['hits']['hits']:
                results.append({
                    'score': hit['_score'],
                    'question': hit['_source']['question'],
                    'answer': hit['_source']['answer'],
                    'search_type': 'semantic'
                })
            
            return results
            
        except Exception as e:
            print(f"Error in semantic search: {e}")
            return []
    
    def perform_keyword_search(self, query: str, k: int = 10) -> List[Dict]:
        """Perform traditional keyword search"""
        try:
            search_body = {
                'size': k,
                'query': {
                    'match': {
                        'question': {
                            'query': query,
                            'operator': 'or'
                        }
                    }
                },
                '_source': ['question', 'answer']
            }
            
            response = self.os_client.search(index=self.index_name, body=search_body)
            
            results = []
            for hit in response['hits']['hits']:
                results.append({
                    'score': hit['_score'],
                    'question': hit['_source']['question'],
                    'answer': hit['_source']['answer'],
                    'search_type': 'keyword'
                })
            
            return results
            
        except Exception as e:
            print(f"Error in keyword search: {e}")
            return []
    
    def perform_hybrid_search(self, query: str, k: int = 10) -> List[Dict]:
        """Combine semantic and keyword search results"""
        semantic_results = self.perform_semantic_search(query, k//2)
        keyword_results = self.perform_keyword_search(query, k//2)
        
        # Combine and deduplicate results
        all_results = semantic_results + keyword_results
        
        # Simple deduplication based on question text
        seen_questions = set()
        unique_results = []
        
        for result in all_results:
            if result['question'] not in seen_questions:
                seen_questions.add(result['question'])
                result['search_type'] = 'hybrid'
                unique_results.append(result)
        
        return unique_results[:k]
    
    def generate_llm_response(self, query: str, search_results: List[Dict], intent_analysis: Dict) -> str:
        """Generate natural language response using LLM"""
        
        # Prepare context from search results
        context_parts = []
        for i, result in enumerate(search_results[:5], 1):
            context_parts.append(f"Result {i}:\nQ: {result['question']}\nA: {result['answer']}\n")
        
        context = "\n".join(context_parts)
        
        # Add conversation history if this is a follow-up
        history_context = ""
        if intent_analysis['is_followup'] and self.conversation_history:
            recent_history = self.conversation_history[-2:]
            history_context = "\n\nPrevious conversation:\n"
            for hist in recent_history:
                history_context += f"User: {hist['query']}\nAssistant: {hist['response'][:200]}...\n\n"
        
        try:
            # Use a simple fallback response generation for demo purposes
            # In a real implementation, you would call AWS Bedrock here
            
            if search_results:
                top_result = search_results[0]
                response = f"Based on the search results, here's what I found:\n\n"
                response += f"**Most Relevant Result:**\n"
                response += f"Q: {top_result['question']}\n"
                response += f"A: {top_result['answer']}\n\n"
                
                if len(search_results) > 1:
                    response += f"I found {len(search_results)} related results. "
                    if intent_analysis['search_type'] == 'semantic':
                        response += "These were found using semantic search to understand the meaning of your question."
                    elif intent_analysis['search_type'] == 'keyword':
                        response += "These were found using keyword matching."
                    else:
                        response += "These were found using a combination of semantic and keyword search."
                
                return response
            else:
                return "I couldn't find specific results for your query. Could you try rephrasing your question or asking about a different topic?"
                
        except Exception as e:
            print(f"Error generating LLM response: {e}")
            # Fallback to simple response
            if search_results:
                return f"I found {len(search_results)} results for your query. The top result is: {search_results[0]['answer'][:200]}..."
            else:
                return "I couldn't find specific results for your query."
    
    def process_query(self, query: str) -> Dict[str, Any]:
        """Main method to process a user query and return agent response"""
        
        print(f"🔍 Processing query: '{query}'")
        
        # Step 1: Analyze query intent
        intent_analysis = self.analyze_query_intent(query)
        print(f"   Intent Analysis: {intent_analysis['search_type']} search, followup: {intent_analysis['is_followup']}")
        
        # Step 2: Perform appropriate search
        if intent_analysis['search_type'] == 'semantic':
            search_results = self.perform_semantic_search(query)
        elif intent_analysis['search_type'] == 'keyword':
            search_results = self.perform_keyword_search(query)
        else:  # hybrid
            search_results = self.perform_hybrid_search(query)
        
        print(f"   Found {len(search_results)} search results")
        
        # Step 3: Generate intelligent response
        response = self.generate_llm_response(query, search_results, intent_analysis)
        
        # Step 4: Store conversation history
        conversation_entry = {
            'timestamp': datetime.now().isoformat(),
            'query': query,
            'intent_analysis': intent_analysis,
            'search_results_count': len(search_results),
            'response': response
        }
        
        self.conversation_history.append(conversation_entry)
        
        return {
            'query': query,
            'response': response,
            'intent_analysis': intent_analysis,
            'search_results': search_results[:3],  # Return top 3 for reference
            'timestamp': conversation_entry['timestamp']
        }

print("✅ SearchAgent class defined successfully!")

### 6. Initialize the Search Agent

Now let's create an instance of our search agent and verify it can connect to OpenSearch.

In [ ]:
# Initialize the search agent
try:
    agent = SearchAgent(
        opensearch_client=aos_client,
        bedrock_client=bedrock_client,
        sentence_model=model,
        index_name='nlp_pqa'
    )
    
    print("✅ Search Agent initialized successfully!")
    
    # Test OpenSearch connection
    try:
        index_info = aos_client.indices.get(index='nlp_pqa')
        print(f"✅ Connected to OpenSearch index 'nlp_pqa'")
        
        # Get document count
        count_response = aos_client.count(index='nlp_pqa')
        doc_count = count_response['count']
        print(f"   Index contains {doc_count} documents")
        
    except Exception as e:
        print(f"⚠️ Could not access OpenSearch index: {e}")
        print("Please ensure you have completed Module 3 to set up the semantic search index.")
        
except Exception as e:
    print(f"❌ Error initializing Search Agent: {e}")
    print("Please check your AWS credentials and OpenSearch configuration.")

### 7. Interactive Agent Demo

Let's demonstrate our agent's capabilities with various types of queries.

In [ ]:
# Demo 1: Basic Product Question
print("=== Demo 1: Basic Product Question ===")
result1 = agent.process_query("Does this work with Xbox?")

print(f"\n🤖 Agent Response:")
print(result1['response'])
print(f"\n📊 Search Strategy: {result1['intent_analysis']['search_type']}")
print(f"📈 Results Found: {len(result1['search_results'])}")

In [ ]:
# Demo 2: Follow-up Question (tests context awareness)
print("\n=== Demo 2: Follow-up Question ===")
result2 = agent.process_query("What about PlayStation?")

print(f"\n🤖 Agent Response:")
print(result2['response'])
print(f"\n📊 Search Strategy: {result2['intent_analysis']['search_type']}")
print(f"📈 Results Found: {len(result2['search_results'])}")
print(f"🔗 Is Follow-up: {result2['intent_analysis']['is_followup']}")

In [ ]:
# Demo 3: Comparison Query (triggers hybrid search)
print("\n=== Demo 3: Comparison Query ===")
result3 = agent.process_query("Compare wireless vs wired headphones for gaming")

print(f"\n🤖 Agent Response:")
print(result3['response'])
print(f"\n📊 Search Strategy: {result3['intent_analysis']['search_type']}")
print(f"📈 Results Found: {len(result3['search_results'])}")

In [ ]:
# Demo 4: Specific Technical Query
print("\n=== Demo 4: Technical Question ===")
result4 = agent.process_query("How do I connect Bluetooth headphones to my device?")

print(f"\n🤖 Agent Response:")
print(result4['response'])
print(f"\n📊 Search Strategy: {result4['intent_analysis']['search_type']}")
print(f"📈 Results Found: {len(result4['search_results'])}")

### 8. View Conversation History

Let's examine how our agent maintains conversation context.

In [ ]:
print("=== Conversation History ===")
print(f"Total interactions: {len(agent.conversation_history)}\n")

for i, entry in enumerate(agent.conversation_history, 1):
    print(f"Interaction {i}:")
    print(f"  🕐 Time: {entry['timestamp']}")
    print(f"  👤 User: {entry['query']}")
    print(f"  🔍 Search: {entry['intent_analysis']['search_type']}")
    print(f"  📊 Results: {entry['search_results_count']}")
    print(f"  🔗 Follow-up: {entry['intent_analysis']['is_followup']}")
    print(f"  🤖 Response: {entry['response'][:150]}...\n")

### 9. Interactive Agent Session

Now you can interact with the agent directly! Run the cell below and ask your own questions.

In [ ]:
def interactive_agent_session():
    """Interactive session with the search agent"""
    print("🤖 Welcome to the Interactive Search Agent!")
    print("Type your questions about products, or 'quit' to exit.\n")
    
    while True:
        try:
            user_query = input("👤 You: ").strip()
            
            if user_query.lower() in ['quit', 'exit', 'bye']:
                print("🤖 Agent: Goodbye! Thanks for using the search agent.")
                break
            
            if not user_query:
                print("🤖 Agent: Please enter a question about products.")
                continue
            
            # Process the query
            result = agent.process_query(user_query)
            
            print(f"\n🤖 Agent: {result['response']}")
            print(f"\n📊 (Used {result['intent_analysis']['search_type']} search with {len(result['search_results'])} results)\n")
            
        except KeyboardInterrupt:
            print("\n🤖 Agent: Session ended by user.")
            break
        except Exception as e:
            print(f"🤖 Agent: Sorry, I encountered an error: {e}")

# Uncomment the line below to start an interactive session
# interactive_agent_session()

print("💡 Tip: Uncomment the line above to start an interactive session with the agent!")

### 10. Performance Analysis

Let's analyze how different search strategies perform for various query types.

In [ ]:
# Test queries to evaluate different search strategies
test_queries = [
    "Does this work with Xbox?",
    "wireless headphones battery life",
    "How to connect Bluetooth?",
    "gaming headset microphone quality",
    "compatible with iPhone",
    "noise cancellation effectiveness",
    "What's the sound quality like?",
    "charging time for wireless earbuds"
]

print("=== Performance Analysis ===")
print("Testing different search strategies on sample queries...\n")

performance_data = []

for query in test_queries:
    print(f"Testing: '{query}'")
    
    # Test semantic search
    start_time = time.time()
    semantic_results = agent.perform_semantic_search(query, k=5)
    semantic_time = time.time() - start_time
    
    # Test keyword search
    start_time = time.time()
    keyword_results = agent.perform_keyword_search(query, k=5)
    keyword_time = time.time() - start_time
    
    # Test hybrid search
    start_time = time.time()
    hybrid_results = agent.perform_hybrid_search(query, k=5)
    hybrid_time = time.time() - start_time
    
    performance_data.append({
        'query': query,
        'semantic_results': len(semantic_results),
        'semantic_time': semantic_time,
        'keyword_results': len(keyword_results),
        'keyword_time': keyword_time,
        'hybrid_results': len(hybrid_results),
        'hybrid_time': hybrid_time
    })
    
    print(f"  Semantic: {len(semantic_results)} results in {semantic_time:.3f}s")
    print(f"  Keyword:  {len(keyword_results)} results in {keyword_time:.3f}s")
    print(f"  Hybrid:   {len(hybrid_results)} results in {hybrid_time:.3f}s\n")

# Create performance summary
df_performance = pd.DataFrame(performance_data)
print("=== Performance Summary ===")
print(f"Average Results per Search Type:")
print(f"  Semantic: {df_performance['semantic_results'].mean():.1f} results")
print(f"  Keyword:  {df_performance['keyword_results'].mean():.1f} results")
print(f"  Hybrid:   {df_performance['hybrid_results'].mean():.1f} results")

print(f"\nAverage Response Time:")
print(f"  Semantic: {df_performance['semantic_time'].mean():.3f}s")
print(f"  Keyword:  {df_performance['keyword_time'].mean():.3f}s")
print(f"  Hybrid:   {df_performance['hybrid_time'].mean():.3f}s")

### 11. Summary and Next Steps

Congratulations! You've successfully built and demonstrated an intelligent search agent with OpenSearch. Let's summarize what we've accomplished and explore next steps.

In [ ]:
print("🎉 Module 6 - Agent Search with OpenSearch Complete!")
print("\n" + "="*60)

print("\n📋 What We Built:")
print("   ✅ Intelligent Search Agent with natural language understanding")
print("   ✅ Multi-strategy search (semantic, keyword, hybrid)")
print("   ✅ Conversation context and follow-up handling")
print("   ✅ Query intent analysis and routing")
print("   ✅ Interactive demonstration interface")
print("   ✅ Performance analysis and insights")

print("\n🔧 Key Technologies Used:")
print("   • Amazon OpenSearch Service for vector and text search")
print("   • Sentence Transformers for embedding generation")
print("   • AWS Bedrock (framework ready) for LLM responses")
print("   • Custom agent logic for intelligent query routing")
print("   • Conversation history and context management")

print("\n🚀 Next Steps and Enhancements:")
print("   1. Integrate AWS Bedrock for production LLM responses")
print("   2. Add voice interface using Amazon Polly/Transcribe")
print("   3. Implement user authentication and personalization")
print("   4. Add product recommendation capabilities")
print("   5. Create a web interface for the agent")
print("   6. Implement feedback learning and model fine-tuning")
print("   7. Add multi-modal search (images, audio)")
print("   8. Scale to handle multiple users concurrently")

print("\n💡 Real-World Applications:")
print("   • Customer service chatbots")
print("   • E-commerce product discovery")
print("   • Knowledge base search")
print("   • Technical documentation assistance")
print("   • Educational content discovery")

print("\n🎯 Key Learning Outcomes:")
print("   • How to build conversational search interfaces")
print("   • Combining multiple search strategies intelligently")
print("   • Implementing context-aware AI agents")
print("   • Performance optimization for real-time search")
print("   • User experience design for AI-powered applications")

if agent.conversation_history:
    print(f"\n📊 Your Session Stats:")
    print(f"   • Total queries processed: {len(agent.conversation_history)}")
    followup_count = sum(1 for entry in agent.conversation_history if entry['intent_analysis']['is_followup'])
    print(f"   • Follow-up queries: {followup_count}")

print("\n" + "="*60)
print("Thank you for completing the Agent Search tutorial!")
print("You now have the foundation to build intelligent, conversational search experiences.")